In [ ]:
#!pip install pydrive

In [ ]:
import os

In [ ]:
contenido = os.listdir('D:/utea/DEV/utea_scripts/reportes_credito_siembra/pdfs')

dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
dic_files

#  CARGAR FILES A DRIVE

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

In [ ]:
print(len(dic_files), 'INFORMES')
dic_files

In [ ]:
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "2DA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile('D:/utea/DEV/utea_scripts/reportes_credito_siembra/pdfs/' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

In [ ]:
print(len(dic_urls), 'URLs')
dic_urls

# CONECTAR A AMIGOCLOUD

In [ ]:
from amigocloud import AmigoCloud

In [ ]:
# CONECTAR A AMIGOCLOUD
acceso = open('../../../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [ ]:
for key in dic_urls:
    query = {'query': 'update dataset_282969 set link_doc_2da = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/31360/sql', query)
    print(key)
print('end')

# -----------------------------------------------------------

# CONECCION A DRIVE

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

In [ ]:
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "REPORTES_CREDITO"):
      fileID = file['id']

file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
file1.SetContentFile("file.pdf")
file1.Upload() # Upload the file.
print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))

In [ ]:
permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})

print(file1['alternateLink'])  # Display the sharable link.

In [ ]:
for file in fileList[0]:
    print(file, '       ',fileList[0][file])

In [ ]:
fileList[0]['title']